In [ ]:
# Import Library
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Load Data
df = pd.read_csv('data.csv')

In [ ]:
# Data Cleansing
# Assuming 'order_datetime' and 'user_last_login_datetime' are in datetime format
df['order_datetime'] = pd.to_datetime(df['order_datetime'])
df['user_last_login_datetime'] = pd.to_datetime(df['user_last_login_datetime'])
df['rejected_datetime'] = pd.to_datetime(df['rejected_datetime'])

In [ ]:
# EDA (Exploratory Data Analysis)
# Display basic statistics
print(df.describe())

In [ ]:
# Data Visualization
# A.1 Top 5 Category
top_categories = df[df['rejected_datetime'].isnull()].groupby('cat_name').size().nlargest(5)
top_categories.plot(kind='bar', rot=45, color='skyblue')
plt.title('Top 5 Categories by Total Orders')
plt.xlabel('Category Name')
plt.ylabel('Total Orders')
plt.show()

Total GMV')
plt.show()

In [ ]:
# A.2 Year of Year Trends
df['segment'] = df['cat_name'].map({
    'Agriculture & Food': 'F&B',
    'Horeca': 'F&B',
    'Machinery & Industrial Parts': 'MRO',
    'Building Materials': 'MRO',
    'Automotive & Transportation': 'MRO',
    'Health & Medical': 'Health & Beauty',
    'Beauty, Sport & Fashion': 'Health & Beauty',
    'Computer & Communication': '3C & Others'
})

tabel_pre_yoy = df[(df['user_last_login_datetime'] >= '2018-01-01') & (df['user_last_login_datetime'] <= '2021-12-31')]
tabel_pre_yoy = tabel_pre_yoy.groupby(['segment', df['order_datetime'].dt.year]).agg(total_gmv=('gmv', 'sum')).reset_index()

tabel_pre_yoy['pct_change_yoy'] = tabel_pre_yoy.groupby('segment')['total_gmv'].pct_change() * 100

plt.figure(figsize=(12, 6))
sns.lineplot(x='order_datetime', y='total_gmv', hue='segment', data=tabel_pre_yoy)
plt.title('Year-over-Year GMV Trends by Segment')
plt.xlabel('Year')
plt.ylabel('

In [ ]:
# B.1 Success Order Rate based on Category
success_order_rate = df.groupby('cat_name').agg(
    total_order=('order_id', 'count'),
    successed_order=('rejected_datetime', lambda x: x.isnull().sum()),
    canceled_order=('rejected_datetime', lambda x: x.notnull().sum())
)

success_order_rate['success_order_rate'] = (success_order_rate['successed_order'] / success_order_rate['total_order']) * 100

success_order_rate = success_order_rate.sort_values(by='success_order_rate', ascending=False).reset_index()

plt.figure(figsize=(12, 6))
sns.barplot(x='cat_name', y='success_order_rate', data=success_order_rate, color='lightgreen')
plt.title('Success Order Rate by Category')
plt.xlabel('Category Name')
plt.ylabel('Success Order Rate (%)')
plt.xticks(rotation=45, ha='right')
plt.show()

In [ ]:
# B.2 Success Order Rate based on Category and Order Platform Source Class
df['platform_source_class'] = df['platform_source'].map({
    'website': 'Website',
    'CMS': 'Website',
    'PWA': 'Website',
    'agent': 'Mobile',
    'ios': 'Mobile',
    'Android': 'Mobile'
})

success_order_rate_platform = df.groupby(['cat_name', 'platform_source_class']).agg(
    total_order=('order_id', 'count'),
    successed_order=('rejected_datetime', lambda x: x.isnull().sum()),
    canceled_order=('rejected_datetime', lambda x: x.notnull().sum())
)

success_order_rate_platform['success_order_rate'] = (
        success_order_rate_platform['successed_order'] / success_order_rate_platform['total_order']) * 100

success_order_rate_platform = success_order_rate_platform.sort_values(by='success_order_rate', ascending=False).reset_index()

plt.figure(figsize=(12, 6))
sns.barplot(x='cat_name', y='success_order_rate', hue='platform_source_class', data=success_order_rate_platform,
            palette='viridis')
plt.title('Success Order Rate by Category and Platform Source Class')
plt.xlabel('Category Name')
plt.ylabel('Success Order Rate (%)')
plt.xticks(rotation=45, ha='right')
plt.legend(title='Platform Source Class')
plt.show()

In [ ]:
# C.1 Buyer Count based on Bucket Size GMV and Total Transaction
df['lifetime_gmv_category'] = pd.cut(df['gmv'].astype(float), bins=[float('-inf'), 500000000, 1000000000, 2000000000, 3000000000, float('inf')],
                                    labels=['<500 Million IDR', '500 Million - 1 Billion IDR',
                                            '1 - 2 Billion IDR', '2 - 3 Billion IDR', '>3 Billion IDR'])

transaction_counts = df.groupby('user_id')['order_id'].count().reset_index(name='total_transactions')

buyer_count_bucket = df.merge(transaction_counts, on='user_id').groupby(['segment', 'lifetime_gmv_category']).agg(
    trx1=('total_transactions', lambda x: x.lt(10).sum()),
    trx2=('total_transactions', lambda x: x.between(10, 20).sum()),
    trx3=('total_transactions', lambda x: x.between(21, 30).sum()),
    trx4=('total_transactions', lambda x: x.between(31, 40).sum()),
    trx5=('total_transactions', lambda x: x.gt(40).sum())
).reset_index()

plt.figure(figsize=(12, 6))
sns.barplot(x='lifetime_gmv_category', y='total_transactions', hue='segment', data=buyer_count_bucket)
plt.title('Buyer Count Based on Bucket Size GMV and Total Transaction')
plt.xlabel('Lifetime GMV Category')
plt.ylabel('Total Transactions')
plt.show()

In [ ]:
# D.1 Active Buyer List in Recent 6 Months
lastest_order = df.groupby('user_id').agg(
    last_order_date=('order_datetime', 'max'),
    top_frequently_category_order_lifetime=('cat_name', lambda x: x.mode().iat[0]),
    total_order=('order_id', 'nunique'),
    total_gmv_lifetime=('gmv', 'sum')
).reset_index()

category_table = df.groupby(['user_id', 'order_datetime', 'cat_name']).size().reset_index(name='category_order')
category_table = category_table.sort_values('order_datetime', ascending=False).groupby('user_id').head(1)

active_buyer_list = lastest_order.merge(category_table[['user_id', 'last_order_date', 'cat_name']], on='user_id',
                                       how='inner')

active_buyer_list['last_order_date_class'] = pd.cut(
    (pd.to_datetime('2022-12-31') - active_buyer_list['last_order_date']).dt.days,
    bins=[-1, 7, 30, 60, 90, 180],
    labels=['< 1 week', '< 1 month', '< 2 months', '< 3 months', '< 6 months']
)

plt.figure(figsize=(12, 6))
sns.countplot(x='last_order_date_class', data=active_buyer_list, palette='pastel')
plt.title('Active Buyer List in Recent 6 Months')
plt.xlabel('Last Order Date Class')
plt.ylabel('Number of Active Buyers')
plt.show()

In [ ]:
# D.2 Inactive Buyer List in Recent 6 Months
inactive_buyer_list = lastest_order[
    lastest_order['last_order_date'] < (pd.to_datetime('2022-12-31') - pd.DateOffset(months=6))
]

inactive_buyer_list['last_login_date_class'] = pd.cut(
    (pd.to_datetime('2022-12-31') - inactive_buyer_list['last_login_datetime']).dt.days,
    bins=[540, 630, 720, 810, 900, 990, 1080, float('inf')],
    labels=['> 24 months', '> 21 months', '> 18 months', '> 15 months', '> 12 months', '> 9 months', '> 6 months']
)

plt.figure(figsize=(12, 6))
sns.countplot(x='last_login_date_class', data=inactive_buyer_list, palette='pastel')
plt.title('Inactive Buyer List in Recent 6 Months')
plt.xlabel('Last Login Date Class')
plt.ylabel('Number of Inactive Buyers')
plt.show()